# Overview
- noglobal デコレータを使う
- jupyter notebook はglobal変数の汚染がすごいのでそれを解決したい。
- twitterなどで少し議論があり、解決策が提案された
    - https://twitter.com/raven_38_/status/1335500447168118784?s=20

# Function

In [16]:
import numpy as np

In [21]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

# Experiment

<br>

関数内でglobal変数が利用される例。  
バグの温床になりやすい。

In [14]:
hoge = 100
def test01(a, b=10):
    print(hoge + a + b)

test01(2)

112


---> test01関数内でglobal変数が使われようとした時エラー表示になってほしい。  
---> そこで、noglobalデコレータの出番。

<br>

使い方は簡単で、関数の上に `@noglobal` をつけたすだけ

In [18]:
hoge = 100
@noglobal
def test01(a, b=10):
    print(hoge + a + b)

test01(2)

NameError: name 'hoge' is not defined

---> ちゃんとエラー出た ;)